<a href="https://colab.research.google.com/github/aditya-2007/MomentumScreener/blob/develop/1%2C3%2C6m_MomentumScreener_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy yfinance pytesseract pillow

In [1]:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.table import Table

# List of tickers - Nifty 200 list
'''tickers = ['ABB.NS','ACC.NS','APLAPOLLO.NS','AUBANK.NS','ADANIENSOL.NS','ADANIENT.NS','ADANIGREEN.NS','ADANIPORTS.NS','ADANIPOWER.NS','ATGL.NS','ABCAPITAL.NS','ABFRL.NS','ALKEM.NS','AMBUJACEM.NS','APOLLOHOSP.NS','APOLLOTYRE.NS',
           'ASHOKLEY.NS','ASIANPAINT.NS','ASTRAL.NS','AUROPHARMA.NS','DMART.NS','AXISBANK.NS','BSE.NS','BAJAJ-AUTO.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BAJAJHLDNG.NS','BALKRISIND.NS','BANDHANBNK.NS','BANKBARODA.NS','BANKINDIA.NS'
           ,'MAHABANK.NS','BDL.NS','BEL.NS','BHARATFORG.NS','BHEL.NS','BPCL.NS','BHARTIARTL.NS','BHARTIHEXA.NS','BIOCON.NS','BOSCHLTD.NS','BRITANNIA.NS','CGPOWER.NS','CANBK.NS','CHOLAFIN.NS','CIPLA.NS','COALINDIA.NS','COCHINSHIP.NS',
           'COFORGE.NS','COLPAL.NS','CONCOR.NS','CUMMINSIND.NS','DLF.NS','DABUR.NS','DELHIVERY.NS','DIVISLAB.NS','DIXON.NS','DRREDDY.NS','EICHERMOT.NS','ESCORTS.NS','EXIDEIND.NS','NYKAA.NS','FEDERALBNK.NS','FACT.NS','GAIL.NS',
           'GMRAIRPORT.NS','GODREJCP.NS','GODREJPROP.NS','GRASIM.NS','HCLTECH.NS','HDFCAMC.NS','HDFCBANK.NS','HDFCLIFE.NS','HAVELLS.NS','HEROMOTOCO.NS','HINDALCO.NS','HAL.NS','HINDPETRO.NS','HINDUNILVR.NS','HINDZINC.NS','HUDCO.NS',
           'ICICIBANK.NS','ICICIGI.NS','ICICIPRULI.NS','IDBI.NS','IDFCFIRSTB.NS','IRB.NS','ITC.NS','INDIANB.NS','INDHOTEL.NS','IOC.NS','IOB.NS','IRCTC.NS','IRFC.NS','IREDA.NS','IGL.NS','INDUSTOWER.NS','INDUSINDBK.NS','NAUKRI.NS','INFY.NS',
           'INDIGO.NS','JSWENERGY.NS','JSWINFRA.NS','JSWSTEEL.NS','JINDALSTEL.NS','JIOFIN.NS','JUBLFOOD.NS','KPITTECH.NS','KALYANKJIL.NS','KOTAKBANK.NS','LTF.NS','LICHSGFIN.NS','LTIM.NS','LT.NS','LICI.NS','LUPIN.NS','MRF.NS','LODHA.NS',
           'M&MFIN.NS','M&M.NS','MRPL.NS','MANKIND.NS','MARICO.NS','MARUTI.NS','MFSL.NS','MAXHEALTH.NS','MAZDOCK.NS','MPHASIS.NS','MUTHOOTFIN.NS','NHPC.NS','NLCINDIA.NS','NMDC.NS','NTPC.NS','NESTLEIND.NS','OBEROIRLTY.NS','ONGC.NS','OIL.NS',
           'PAYTM.NS','OFSS.NS','POLICYBZR.NS','PIIND.NS','PAGEIND.NS','PATANJALI.NS','PERSISTENT.NS','PETRONET.NS','PHOENIXLTD.NS','PIDILITIND.NS','POLYCAB.NS','POONAWALLA.NS','PFC.NS','POWERGRID.NS','PRESTIGE.NS','PNB.NS','RECLTD.NS',
           'RVNL.NS','RELIANCE.NS','SBICARD.NS','SBILIFE.NS','SJVN.NS','SRF.NS','MOTHERSON.NS','SHREECEM.NS','SHRIRAMFIN.NS','SIEMENS.NS','SOLARINDS.NS','SONACOMS.NS','SBIN.NS','SAIL.NS','SUNPHARMA.NS','SUNDARMFIN.NS','SUPREMEIND.NS','SUZLON.NS',
           'TVSMOTOR.NS','TATACHEM.NS','TATACOMM.NS','TCS.NS','TATACONSUM.NS','TATAELXSI.NS','TATAMOTORS.NS','TATAPOWER.NS','TATASTEEL.NS','TATATECH.NS','TECHM.NS','TITAN.NS','TORNTPHARM.NS','TORNTPOWER.NS','TRENT.NS','TIINDIA.NS','UPL.NS',
           'ULTRACEMCO.NS','UNIONBANK.NS','UNITDSPR.NS','VBL.NS','VEDL.NS','IDEA.NS','VOLTAS.NS','WIPRO.NS','YESBANK.NS','ZOMATO.NS','ZYDUSLIFE.NS']'''
tickers = ['ABB.NS','ACC.NS']

# Function to fetch historical data
def get_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, multi_level_index=False)

# Set dates
end_date = datetime.today()
start_date = end_date - timedelta(days=365 * 2)  # 2 years of data for moving averages

# Data dictionary to hold stock data
data = {}

# Fetch data for all tickers
for ticker in tickers:
    try:
        stock_data = get_data(ticker, start_date, end_date)
        print(stock_data.columns)
        if len(stock_data) > 0:
            data[ticker] = stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Create a DataFrame for summary
summary = []

# Analyze each stock
for ticker, df in data.items():
    try:
        # Calculate EMAs
        df['EMA100'] = df['Adj Close'].ewm(span=100).mean()

        # Last 1-year return
        one_year_return = (df['Adj Close'].iloc[-1] / df['Adj Close'].iloc[-252] - 1) * 100

        # 52-week high
        high_52_week = df['Adj Close'].iloc[-252:].max()
        within_20_pct_high = df['Adj Close'].iloc[-1] >= high_52_week * 0.8

        # More than 45% up days in the last 6 months (126 trading days)
        six_month_data = df['Adj Close'].iloc[-126:]
        up_days = (six_month_data.pct_change() > 0).sum()
        up_days_pct = up_days / len(six_month_data) * 100

        # Filtering criteria
        if (df['Adj Close'].iloc[-1] >= df['EMA100'].iloc[-1] and
            one_year_return >= 6.5 and
            within_20_pct_high and
            up_days_pct > 45):

            # Calculate returns
            return_6m = (df['Adj Close'].iloc[-1] / df['Adj Close'].iloc[-126] - 1) * 100
            return_3m = (df['Adj Close'].iloc[-1] / df['Adj Close'].iloc[-63] - 1) * 100
            return_1m = (df['Adj Close'].iloc[-1] / df['Adj Close'].iloc[-21] - 1) * 100

            summary.append({
                'Ticker': ticker,
                'Return_6M': return_6m,
                'Return_3M': return_3m,
                'Return_1M': return_1m,
            })
    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")

# Convert summary to DataFrame
df_summary = pd.DataFrame(summary)
print(df_summary)

# Round off returns to 1 decimal place
df_summary['Return_6M'] = df_summary['Return_6M'].round(1)
df_summary['Return_3M'] = df_summary['Return_3M'].round(1)
df_summary['Return_1M'] = df_summary['Return_1M'].round(1)

# Ranking based on returns
df_summary['Rank_6M'] = df_summary['Return_6M'].rank(ascending=False)
df_summary['Rank_3M'] = df_summary['Return_3M'].rank(ascending=False)
df_summary['Rank_1M'] = df_summary['Return_1M'].rank(ascending=False)

# Calculate final rank
df_summary['Final_Rank'] = df_summary['Rank_6M'] + df_summary['Rank_3M'] + df_summary['Rank_1M']

# Sort by final rank and get top 30
df_summary_sorted = df_summary.sort_values('Final_Rank').head(40)

# Assign position based on final rank
df_summary_sorted['Position'] = np.arange(1, len(df_summary_sorted) + 1)

# Separate top 15 and next 15
top_15 = df_summary_sorted.head(15)
next_15 = df_summary_sorted.iloc[15:30]

# Prepare table data
tbl_data = []
for i, row in df_summary_sorted.iterrows():
    tbl_data.append([row['Ticker'],
                     row['Return_6M'], row['Rank_6M'],
                     row['Return_3M'], row['Rank_3M'],
                     row['Return_1M'], row['Rank_1M'],
                     row['Final_Rank'], row['Position']])

# Plot the results using matplotlib
fig, ax = plt.subplots(figsize=(14, 10))
ax.axis('tight')
ax.axis('off')

# Create a table
col_labels = ['Ticker', 'Return_6M', 'Rank_6M', 'Return_3M', 'Rank_3M', 'Return_1M', 'Rank_1M', 'Final_Rank','Position']
table = ax.table(cellText=tbl_data, colLabels=col_labels, cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1, 1.5)

# Draw a red line to separate the top 15
for i in range(len(col_labels)):
    cell = table[(15, i)]
    cell.set_edgecolor('red')
    cell.set_linewidth(3)

# Adjust layout
plt.subplots_adjust(left=0.1, top=0.95)  # Adjust top parameter for title position
#plt.title('Top 40 Stocks Momentum Ranking', fontsize=18, y=1.05)  # y parameter for title position
plt.show()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')


[*********************100%***********************]  1 of 1 completed


Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
Error analyzing ABB.NS: 'Adj Close'
Error analyzing ACC.NS: 'Adj Close'
Error analyzing APLAPOLLO.NS: 'Adj Close'
Error analyzing AUBANK.NS: 'Adj Close'
Error analyzing ADANIENSOL.NS: 'Adj Close'
Error analyzing ADANIENT.NS: 'Adj Close'
Error analyzing ADANIGREEN.NS: 'Adj Close'
Error analyzing ADANIPORTS.NS: 'Adj Close'
Error analyzing ADANIPOWER.NS: 'Adj Close'
Error analyzing ATGL.NS: 'Adj Close'
Error analyzing ABCAPITAL.NS: 'Adj Close'
Error analyzing ABFRL.NS: 'Adj Close'
Error analyzing ALKEM.NS: 'Adj Close'
Error analyzing AMBUJACEM.NS: 'Adj Close'
Error analyzing APOLLOHOSP.NS: 'Adj Close'
Error analyzing APOLLOTYRE.NS: 'Adj Close'
Error analyzing ASHOKLEY.NS: 'Adj Close'
Error analyzing ASIANPAINT.NS: 'Adj Close'
Error analyzing ASTRAL.NS: 'Adj Close'
Error analyzing AUROPHARMA.NS: 'Adj Close'
Error analyzing DMART.NS: 'Adj Close'
Error analyzing

KeyError: 'Return_6M'